In [1]:
import os
import pandas as pd
from pydub import AudioSegment
import speech_recognition as sr
from speechFileMP3ToText import speechFileMP3ToText


train_file = os.path.join("tsv_files", "train.tsv")
dev_file = os.path.join("tsv_files", "dev.tsv")
test_file = os.path.join("tsv_files", "test.tsv")

train_df = pd.read_csv(train_file, sep='\t')
dev_df = pd.read_csv(dev_file, sep='\t')
test_df = pd.read_csv(test_file, sep='\t')

# print(train_df)
# print(test_df)
# train_df
# test_df

#------------------------------------------------------------------------
#Extracting the rows and creating new files with the interesting sentences

# List of sentence values to filter
sentences_to_keep = ["yes", "no", "one", "two", "three", "four"]

# Filter the rows of the train_df DataFrame
filtered_train_df = train_df[train_df["sentence"].isin(sentences_to_keep)]

# Filter the rows of the dev_df DataFrame
filtered_dev_df = dev_df[dev_df["sentence"].isin(sentences_to_keep)]

# Filter the rows of the test_df DataFrame
filtered_test_df = test_df[test_df["sentence"].isin(sentences_to_keep)]

# Write the filtered data to new TSV files
filtered_train_df.to_csv("./tsv_files/filtered_train.tsv", sep='\t', index=False)
filtered_dev_df.to_csv("./tsv_files/filtered_dev.tsv", sep='\t', index=False)
filtered_test_df.to_csv("./tsv_files/filtered_test.tsv", sep='\t', index=False)


# filtered_train_df
# filtered_dev_df
filtered_test_df

i=0

for filename in os.listdir('./en_test_subset/'):
    i+=1
    if filename != "target_directory":
        print("i=",i)
        output = speechFileMP3ToText(filename)
        filtered_test_df.loc[filtered_test_df['path'] == filename, 'predicted_sentence'] = output
        filtered_test_df = filtered_test_df.reindex(filtered_test_df.loc[filtered_test_df['path'] == filename].index.union(filtered_test_df.index))
filtered_test_df


# filtered_test_df.size #99

# une nouvelle colonne "correct_predicted" qui est un boolean et qui possède True si les champs "path" et "predicted_sentence" sont égales
#Le problème c'est si on a dans correct_predicted "3" ou "444"
#-->On va utiliser fuzzywuzzy (on a aussi besoin d'installer la python-Levenshtein)
# filtered_test_df['correct_predicted'] = filtered_test_df.apply(lambda row: row['sentence'] == row['predicted_sentence'], axis=1)
from fuzzywuzzy import fuzz
#90 ça ne suffit pas, tester à la main pour voir?
#Même avec 50, ça ne marche pas
# filtered_test_df["correct_predicted"] = filtered_test_df.apply(lambda row: fuzz.token_set_ratio(row['sentence'], row['predicted_sentence']) > 50, axis=1)


#On va utiliser une autre approche
#C'est bcp plus long, ça a pris ~1min
#ça a résolu le problème du "3" mais pas du "444"

def convert_number(n):
    if n == "1":
        return "one"
    elif n == "2":
        return "two"
    elif n == "3":
        return "three"
    elif n == "4":
        return "four"
    else:
        return n

filtered_test_df['predicted_sentence'] = filtered_test_df['predicted_sentence'].apply(convert_number)
filtered_test_df["correct_predicted"] = filtered_test_df.apply(lambda row: row['sentence'] == row['predicted_sentence'], axis=1)

filtered_test_df


i= 1


C:\Users\Simon\AppData\Local\Temp\ipykernel_4648\500016601.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_test_df.loc[filtered_test_df['path'] == filename, 'predicted_sentence'] = output


i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
i= 10


,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment,predicted_sentence,correct_predicted
1,02a86ce07b7b7d4dfb369c632369b6d7ebcff87472beae...,common_voice_en_22005778.mp3,yes,3,0,NaN,NaN,NaN,en,Benchmark,yes,True
2,042bc2389193679673f4860dd38356e5c1bec5bf9b95d6...,common_voice_en_22310890.mp3,yes,3,0,NaN,NaN,NaN,en,Benchmark,yes,True
5,099b4fa0c62e39a22db1e1de8d756cb1eb29f122dfab99...,common_voice_en_21960400.mp3,three,2,1,NaN,NaN,NaN,en,Benchmark,three,True
7,0b5e0bb806dba1eb60c51b25420e40d7daffff30ca29b9...,common_voice_en_22316853.mp3,yes,2,0,NaN,NaN,NaN,en,Benchmark,yes,True
8,0bdbe5114e8f69ee5a4e2b46ed365bd10f7bfa5addbaba...,common_voice_en_22327660.mp3,yes,3,0,NaN,NaN,NaN,en,Benchmark,yes,True
...,...,...,...,...,...,...,...,...,...,...,...,...
8215,c2cf4027a6375de35e9958fbec94e8af6eba58837bdc51...,common_voice_en_21951428.mp3,two,2,0,NaN,NaN,NaN,en,Benchmark,NaN,False
8217,c2cf4027a6375de35e9958fbec94e8af6eba58837bdc51...,common_voice_en_21951430.mp3,no,4,0,NaN,NaN,NaN,en,Benchmark,NaN,False
8219,c30d3c26910df3d2538bf03d190eb3b5bcc2a183b793c8...,common_voice_en_22150155.mp3,yes,2,0,NaN,NaN,NaN,en,Benchmark,NaN,False
8221,c30d3c26910df3d2538bf03d190eb3b5bcc2a183b793c8...,common_voice_en_22150157.mp3,four,2,1,NaN,NaN,NaN,en,Benchmark,NaN,False


In [7]:
top_10_rows = filtered_test_df.iloc[:10]
top_10_rows

,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment,predicted_sentence,correct_predicted
1,02a86ce07b7b7d4dfb369c632369b6d7ebcff87472beae...,common_voice_en_22005778.mp3,yes,3,0,NaN,NaN,NaN,en,Benchmark,yes,True
2,042bc2389193679673f4860dd38356e5c1bec5bf9b95d6...,common_voice_en_22310890.mp3,yes,3,0,NaN,NaN,NaN,en,Benchmark,yes,True
5,099b4fa0c62e39a22db1e1de8d756cb1eb29f122dfab99...,common_voice_en_21960400.mp3,three,2,1,NaN,NaN,NaN,en,Benchmark,three,True
7,0b5e0bb806dba1eb60c51b25420e40d7daffff30ca29b9...,common_voice_en_22316853.mp3,yes,2,0,NaN,NaN,NaN,en,Benchmark,yes,True
8,0bdbe5114e8f69ee5a4e2b46ed365bd10f7bfa5addbaba...,common_voice_en_22327660.mp3,yes,3,0,NaN,NaN,NaN,en,Benchmark,yes,True
9,0fe95db773eb34093ddcae0d35d96d5ea1e87752d4dbb8...,common_voice_en_21875014.mp3,two,2,0,NaN,NaN,NaN,en,Benchmark,<class 'speech_recognition.UnknownValueError'>,False
10,103ad2011d22a72f27d97a76b021ab0bd07183942959de...,common_voice_en_22158989.mp3,two,2,0,NaN,NaN,NaN,en,Benchmark,do,False
12,11c48af86864b1f5a526398ef1024db362cec1bbe3dddd...,common_voice_en_22227005.mp3,four,2,1,NaN,NaN,NaN,en,Benchmark,444,False
18,194603f19cc01339f1fb71012762e4e29d9e2da50caffa...,common_voice_en_22300196.mp3,no,2,0,NaN,NaN,NaN,en,Benchmark,no,True
19,19d7407e16df855d6832409d8288186c5f1eff335349b7...,common_voice_en_22076105.mp3,four,2,1,NaN,NaN,NaN,en,Benchmark,NaN,False
